In [1]:
n=20
k=10
w=3

H=random_matrix(GF(2),n-k,n)
e=zero_vector(GF(2),n)
e[0]=e[1]=e[2]=1
s=H*e

In [2]:
def hybrid_coprocessor(H,s,w,p,alpha,beta):
    e_prime=zero_vector(GF(2),n)
    e_quant=zero_vector(GF(2),n-alpha-beta)
    I_nkb=identity_matrix(GF(2),n-k-beta)
    while True:
        P=Permutations(n).random_element().to_matrix().change_ring(GF(2))
        HP=H*P
        HP=HP.augment(s)
        HP=HP.echelon_form()
        s_prime=HP.column(n)
        H_prime1=HP[[i for i in range(n-k-beta)],[i for i in range(n-k,n-alpha)]]
        H_quant=I_nkb.augment(H_prime1)
        reduced_instance=(H_quant,s_prime[:n-k-beta],w-p)
        
        #JUST FOR DEBUGGING (set e to the solution of the SD problem)
        ep=P.inverse()*e
        if ep[n-k-beta:n-k].hamming_weight()==p:
            e_quant=vector(GF(2),ep.list()[:n-k-beta]+ep.list()[n-k:n-alpha])
        #+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
        #++ solve reduced instance with quantum algorithm, store in e_quant ++
        #+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
        
        eqw=e_quant.hamming_weight()
        if eqw<=w-p and H_quant*e_quant==s_prime[:n-k-beta]:
            H_prime2=HP[[i for i in range(n-k-beta,n-k)],[i for i in range(n-k,n-alpha)]]
            e_beta=s_prime[n-k-beta:n-k]+H_prime2*e_quant[n-k-beta:]
            if eqw+e_beta.hamming_weight()<=w:
                solution=P*vector(GF(2),e_quant[:n-k-beta].list()+e_beta.list()+e_quant[n-k-beta:].list()+[0]*alpha)
                if H*solution==s:
                    return solution
            
        

In [3]:
e2=hybrid_coprocessor(H,s,3,1,1,3)
print(matrix(1,n,e2))

[1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
